In [1]:
!pip install stanza -q

DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\object_detection-0.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\pyparsing-2.4.7-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\vrishank\appdata\local\programs\python\python312\lib\site-packages\sacrebleu-2.2.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pi

In [35]:
import stanza
from nltk import Tree

stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')


2025-06-24 00:53:53 INFO: Downloaded file to C:\Users\Vrishank\stanza_resources\resources.json
2025-06-24 00:53:53 INFO: Downloading default packages for language: en (English) ...
2025-06-24 00:53:53 INFO: File exists: C:\Users\Vrishank\stanza_resources\en\default.zip
2025-06-24 00:53:57 INFO: Finished downloading models and saved to C:\Users\Vrishank\stanza_resources
2025-06-24 00:53:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-06-24 00:53:57 INFO: Downloaded file to C:\Users\Vrishank\stanza_resources\resources.json
2025-06-24 00:53:57 WARNING: Language en package default expects mwt, which has been added
2025-06-24 00:53:58 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2025-06-24 00:53:58 INFO: Using device: cpu
2025-06-24 00:53:58 INFO: Loading: tokenize
2025-06-24 00:53:58 INFO: Loading: mwt
2025-06-24 00:53:58 INFO: Loading: pos
2025-06-24 00:53:59 INFO: Loading: constituency
2025-06-24 00:54:00 INFO: Done loading processors!


In [36]:
doc = nlp("delete files with inode number specified by REGEX under current directory")
tree = Tree.fromstring(str(doc.sentences[0].constituency))
tree.pretty_print()

        ROOT                                                                          
         |                                                                             
         S                                                                            
         |                                                                             
         VP                                                                           
   ______|____                                                                         
  |           NP                                                                      
  |       ____|_______________                                                         
  |      |                    PP                                                      
  |      |     _______________|_______________                                         
  |      |    |                               NP                                      
  |      |    |           ____________

In [69]:
def get_leaf_phrases(tree):
    leaf_positions = tree.treepositions('leaves')
    word_to_phrase = []

    for pos in leaf_positions:
        try:
            # Go up two levels from the leaf
            parent_1 = pos[:-1]       # immediate parent
            parent_2 = pos[:-2]       # two levels up
            phrase_subtree = tree[parent_2]
            phrase = " ".join(phrase_subtree.leaves())
            word = tree[pos]
            word_to_phrase.append((word, phrase, phrase_subtree.label()))
        except IndexError:
            # Skip if not enough levels
            continue

    return word_to_phrase

phrases_data = get_leaf_phrases(tree)

for word, phrase, label in phrases_data:
    print(f"Word: '{word}' → Phrase: '{phrase}' (Type: {label})")


Word: 'delete' → Phrase: 'delete files with inode number specified by REGEX under current directory' (Type: VP)
Word: 'files' → Phrase: 'files' (Type: NP)
Word: 'with' → Phrase: 'with inode number specified by REGEX under current directory' (Type: PP)
Word: 'inode' → Phrase: 'inode number' (Type: NP)
Word: 'number' → Phrase: 'inode number' (Type: NP)
Word: 'specified' → Phrase: 'specified by REGEX under current directory' (Type: VP)
Word: 'by' → Phrase: 'by REGEX' (Type: PP)
Word: 'REGEX' → Phrase: 'REGEX' (Type: NP)
Word: 'under' → Phrase: 'under current directory' (Type: PP)
Word: 'current' → Phrase: 'current directory' (Type: NP)
Word: 'directory' → Phrase: 'current directory' (Type: NP)


In [70]:
phrases = []
for word, phrase, label in phrases_data:
    phrases.append(phrase)

phrases

['delete files with inode number specified by REGEX under current directory',
 'files',
 'with inode number specified by REGEX under current directory',
 'inode number',
 'inode number',
 'specified by REGEX under current directory',
 'by REGEX',
 'REGEX',
 'under current directory',
 'current directory',
 'current directory']

Now that we have that,
* remove the phrases with more than 4 words. 
* add the missing words in the right order. ("delete" is missing here)
* remove duplicate phrases
* remove redundant phrases ("by REGEX" and "REGEX", "current directory" and "under current directory")
* 

In [ ]:
# remove phrases with more than 4 words
for i, phrase in enumerate(phrases):
    if len(phrase.split()) > 4:
        del phrases[i]

phrases

In [73]:
# remove duplicate phrases
for i,phrase1 in enumerate(phrases):
    for j,phrase2 in enumerate(phrases):
        if j <= i:
            continue

        if phrase1 == phrase2:
            del phrases[j]

phrases

['files',
 'inode number',
 'by REGEX',
 'REGEX',
 'under current directory',
 'current directory']

whats left:
* remove redundant phrases ("by REGEX" and "REGEX", "current directory" and "under current directory")
* add the missing words in the right order. ("delete" is missing here)
* 